# Integration of Medication Flag

Goal: Integrate medication flag based on `inputevents_based_medications.parquet` (produced by `inputevents_mv_analysis.ipynb`) indicating whether alarm is more likely to be relevant. An alarm is more likely to be relevant if a medication was given within a certain period of time around the alarm (1h before to 1h after the alarm, aka a time window of 120min around the alarm). This time window can be adjusted with the variable `minutes_around_alarm`.

Note: The merging was necessary because otherwise the two data frames would not be comparable due to their different sizes.

## Reading of Alarm and Medication Data

In [ ]:
import pandas as pd

alarm_data = pd.read_csv('../../data/alarm_data.csv')
alarm_data.CHARTTIME = pd.to_datetime(alarm_data.CHARTTIME)

medication = pd.read_parquet('../../data/inputevents_based_medications.parquet', engine='pyarrow')
medication.STARTTIME = pd.to_datetime(medication.STARTTIME)
medication.ENDTIME = pd.to_datetime(medication.ENDTIME)

## Addition of Column for Medical Flag

In [ ]:
# Add MEDICATION column for flag and set default to 'False' for all alarms
alarm_data['MEDICATION'] = False

## Merging of Alarm and Medication Data

In [ ]:
# Filter for ICU stays that are in medication data
unique_med_stays = medication['ICUSTAY_ID'].unique().tolist()
alarm_data_subset = alarm_data[alarm_data['ICUSTAY_ID'].isin(unique_med_stays)]

# Remove columns that are not needed for comparison after merge
alarm_data_subset = alarm_data_subset[['ROW_ID', 'ICUSTAY_ID', 'CHARTTIME', 'MEDICATION']]
medication = medication[['ICUSTAY_ID', 'STARTTIME', 'ENDTIME']]

# Merge data frames based on ICUSTAY_ID
alarms_with_med = alarm_data_subset.merge(medication, left_on='ICUSTAY_ID', right_on='ICUSTAY_ID')
alarms_with_med.head()

## Setting of Medication Flag

In [ ]:
# Set time window around alarm (in minutes)
# Half of this duration is applied to time before and after an alarm respectively
minutes_around_alarm = 120

# Set medication flag if medication was administered in right time period
alarms_with_med.loc[
    (alarms_with_med['CHARTTIME'] >= alarms_with_med['STARTTIME'] - pd.Timedelta(minutes=minutes_around_alarm/2)) &
    (alarms_with_med['CHARTTIME'] <= alarms_with_med['ENDTIME'] + pd.Timedelta(minutes=minutes_around_alarm/2)),
    'MEDICATION'] = True

## Extraction of Alarm Data with Medication Flag

In [ ]:
# Extract relevant row IDs from merged data frame
relevant_row_ids = alarms_with_med[alarms_with_med['MEDICATION'] == True].ROW_ID.unique().tolist()

# Set medication flag in original alarm data
alarm_data.loc[alarm_data['ROW_ID'].isin(relevant_row_ids), 'MEDICATION'] = True

# Save as parquet file
pd.DataFrame(alarm_data).to_parquet(f'../../data/alarm_data_with_med_{str(minutes_around_alarm)}min.parquet', engine='pyarrow')

## Checking of Overall Amount of Likely Relevant Alarms

In [ ]:
import pandas as pd

alarm_data_with_med_120min = pd.read_parquet('../../data/alarm_data_with_med_120min.parquet', engine='pyarrow')

In [ ]:
# Extract how many alarms were actually relevant
relevant_alarms_120min = alarm_data_with_med_120min[alarm_data_with_med_120min['MEDICATION'] == True]

print(f'# alarms: {len(alarm_data_with_med_120min)}')
print(f'# relevant alarms: {len(relevant_alarms_120min)}')

For 189,415 of the 285,350 alarms, a medication is administered between one hour before and one hour after the alarm. Maybe the choosen time window of 2 hours around an alarm should be reduced.

In [ ]:
import pandas as pd

alarm_data_with_med_60min = pd.read_parquet('../../data/alarm_data_with_med_60min.parquet', engine='pyarrow')

In [ ]:
relevant_alarms_60min = alarm_data_with_med_60min[alarm_data_with_med_60min['MEDICATION'] == True]

print(f'# relevant alarms: {len(relevant_alarms_60min)}')

In [ ]:
relevant_alarms_60min['ICUSTAY_ID'].value_counts().describe()

For 173,151 of the 285,350 alarms, a medication is administered between 30min before and 30min after the alarm which is still a lot. With this time window, there are 1 to 12 relevant alarms in 75% of the ICU stays.